# Домашнее задание № 3. Исправление опечаток

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [1]:
import os, re
from string import punctuation
punctuation += "«»—…“”"
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize

punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter
from tqdm.notebook import tqdm
import textdistance
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [2]:
bad = open('data/sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('data/correct_sents.txt', encoding='utf8').read().splitlines()

In [3]:
corpus = open('data/wiki_data.txt', encoding='utf8').read()
vocab = Counter(re.findall('\w+', corpus.lower()))

word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', max_features=10000, ngram_range=(1,3))
X = vec.fit_transform(vocab)

In [4]:
vec

CountVectorizer(analyzer='char', max_features=10000, ngram_range=(1, 3))

In [5]:
#функции из семинара
def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()
    
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(topn)

def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    
    # вся эффективноть берется из того, что мы сразу считаем близость 
    # 1 вектора ко всей матрице (словам в словаре)
    # считать по отдельности циклом было бы дольше
    # вместо одного вектора может даже целая матрица
    # тогда считаться в итоге будет ещё быстрее
    
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]

def get_closest_hybrid_match_old(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)
   
    return closest

N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N

def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1


In [5]:
#дополненная функция
def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)
    dist = closest[0][1]
    closest_res = Counter() #словарь с кандидатами с одинаковым расстоянием редактирования (лучшим)
    closest_res[closest[0]] = P(closest[0][0])
    i = 1
    while i < len(closest) and closest[i][1] == dist:
        closest_res[closest[i]] = P(closest[i][0]) #каждому кандидату присваиваем вероятность
        i += 1
        
    return closest_res.most_common(1)[0]
    #return closest_res.most_common()

In [6]:
tst = get_closest_hybrid_match('вобще', X, vec)

In [7]:
tst

(('вообще', 0.8333333333333334), 4.228675111473113e-05)

проверим качество


In [8]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

In [130]:
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab):
            pred = cashed.get(pair[1], get_closest_hybrid_match(pair[1], X, vec)[0][0])
            cashed[pair[1]] = pred
        else:
            pred = pair[1]
        
            
        if pred == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1
            
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] != pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                mistaken_fixed += 1

  0%|          | 0/915 [00:00<?, ?it/s]

In [131]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)


0.8556278139069535
0.48835403726708076
0.09004249454461927


In [ ]:
#стало немного лучше 
#(по сравнению с семинаром:
#0.8533266633316658
#0.4704968944099379
#0.09004249454461927)

In [85]:
[(wt[0], wt[1], get_closest_hybrid_match(wt[1],X,vec)) for wt, _ in Counter(mistakes).most_common(10)]

[('сегодня', 'седня', ('средняя', 0.7142857142857143)),
 ('вообще', 'вобще', ('вообще', 0.8333333333333334)),
 ('вообще', 'ваще', ('чаще', 0.75)),
 ('естественно', 'естесственно', ('естественно', 0.9166666666666666)),
 ('хочется', 'хочеться', ('хочется', 0.875)),
 ('кстати', 'кстате', ('кстати', 0.8333333333333334)),
 ('очень', 'ооочень', ('сорочень', 0.75)),
 ('что-то', 'что-то', ('тото', 0.6666666666666667)),
 ('как-то', 'както', ('факто', 0.8)),
 ('очень', 'оооочень', ('сорочень', 0.75))]

## 2.  Symspell (7 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Он основан только на одной операции - удалении символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово  (!) 
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

1) Составляется словарь правильных слов

In [9]:
vocab = Counter(re.findall('\w+', corpus.lower(), flags=re.DOTALL))

2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово (!)

In [10]:
def edits(word):
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes_1l    = [L + R[1:]               for L, R in splits if R]
    return set(deletes_1l)

def edits2(word): 
    "Создаем кандидатов, которые отличаются на две буквы"
    return (e2 for e1 in edits(word) for e2 in edits(e1))

In [11]:
def make_edited_voc(voc):
    edits_voc = Counter()
    for word in voc:
        for e in edits(word):
            if e not in edits_voc:
                edits_voc[e] = [word]
            else:
                edits_voc[e].append(word)
# с удалениями 2 букв               
#        for e in edits2(word):
#             if e not in edits_voc:
#                 edits_voc[e] = [word]
#             else:
#                 edits_voc[e].append(word)
    return edits_voc

In [12]:
edits_voc = make_edited_voc(vocab)

3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  

In [29]:
def correction(word, voc_eds, voc_all): 
    if word in voc_all.keys():
        return word
    #для двух удалений:
    #eds = [w for w in edits(word) if w in voc_eds] + [w for w in edits2(word) if w in voc_eds]
    eds = [w for w in edits(word) if w in voc_eds] #все варианты слова с удалениями
    corrected = Counter()
    for e in eds: #для каждого варианта удаления
        for w in voc_eds[e]: #находим все варианты правильных слов
            corrected[w] = P(w) #записываем "правильные" слова с их вероятностью
    return corrected.most_common(1)[0][0] #выбираем самое вероятное слово

In [36]:
correction('жода', edits_voc, vocab)

'года'

In [32]:
correction('сонце', edits_voc, vocab)

'конце'

In [41]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

unrec = []

total_correct = 0
correct_broken = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        # чтобы два раза не исправлять одно и тоже слово - закешируем его
        # перед тем как считать исправление проверим нет ли его в кеше
        try:
            predicted = cashed.get(pair[1], correction(pair[1], edits_voc, vocab))
            cashed[pair[1]] = predicted
       
        

            if predicted == pair[0]:
                correct += 1
            total += 1

            if pair[0] == pair[1]:
                total_correct += 1
                if pair[0] !=  predicted:
                    correct_broken += 1
            else:
                total_mistaken += 1
                if pair[0] == predicted:
                    mistaken_fixed += 1
        except:
            unrec.append(pair[1])

  0%|          | 0/915 [00:00<?, ?it/s]

In [39]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8932081240445512
0.31076178960096734
0.04897371263953907


In [ ]:
#вроде норм...